In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal,Beta,Dirichlet
from torch.distributions import Categorical
from torch.distributions.normal import Normal
import os
os.environ["NGSIM_DIR"] = "/home/surender/Downloads/NGSIM"
os.environ["OPENDD_DIR"] = "/home/surender/Downloads/openDD"
os.environ["CARLA_PATH"] = "/home/surender/Downloads/carlaOld"
import sys
#sys.path.append('/home/surender/Downloads/CARLA_0.9.9.4/PythonAPI/carla/dist')
import carla
import random
import argparse

from carla_real_traffic_scenarios.carla_maps import CarlaMaps
from carla_real_traffic_scenarios.ngsim import NGSimDatasets, DatasetMode
from carla_real_traffic_scenarios.ngsim.scenario import NGSimLaneChangeScenario
from carla_real_traffic_scenarios.opendd.scenario import OpenDDScenario
from carla_real_traffic_scenarios.reward import RewardType
from carla_real_traffic_scenarios.scenario import Scenario

from carla_birdeye_view import BirdViewProducer, BirdViewCropType, PixelDimensions
from PIL import Image
#from IPython.display import clear_output, Image, display, HTML
import cv2

%matplotlib tk
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import threading
import time
import math
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler
from controller import VehiclePIDController

In [ ]:
#max_score

In [ ]:
random_seed = 76
torch.manual_seed(random_seed)
random.seed(random_seed)
#np.random.seed(random.seed)

In [2]:
if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    
    
    print("Device set to : cpu")

Device set to : GeForce GTX 1060 3GB


In [ ]:
class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.helper = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]
        del self.helper[:]

In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, has_continuous_action_space, action_std_init,hidden_size = 512,num_channels = 5):
        super(ActorCritic, self).__init__()

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_dim = action_dim
            self.action_var = torch.full((action_dim,), action_std_init * action_std_init).to(device)
        
        self.layer1 = nn.LSTM(16, 512,batch_first=True)
            
        #self.mem = (torch.zeroes)
        self.layer2 = nn.Sequential(
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.Dropout(0.4),
            nn.ReLU()
        
        
        )
        self.actor = nn.Sequential(
            
            
            nn.Linear(128, 128),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(128,12),
            nn.Softmax(dim=-1)
            
        )
        self.critic = nn.Sequential(
            nn.Linear(128,1),
            nn.Tanh()
        )
    def act(self, state,val = False):

        if self.has_continuous_action_space:

            inps1 = state

            action_m = self.layer1(inps1)

            action_mean = self.actor(action_m)
            if val:
                return action_mean.detach(),-1
            action_log_std = self.action_log_std.expand_as(action_mean)


            dist = Normal(action_mean,action_log_std.exp())
        else:
            #inps1 = state[:,:-3]
            #inps2 = state[:,-3:]
            action_m,_ = self.layer1(state)
            #print(action_m,inps2,action_m.shape,inps2.shape)
            #inps = torch.cat((action_m,inps2),1)
            action_m = self.layer2(action_m.reshape(1,action_m.shape[-1]))
            action_probs = self.actor(action_m)
            dist = Categorical(action_probs)

        action = dist.sample()

        action_logprob = dist.log_prob(action)
        #action = action*2 - 1

        return action.detach(), action_logprob.detach()
    def evaluate(self, state, action):
        #print("NOTHING")
        
        if self.has_continuous_action_space:
            '''
            if len(state.shape) == 3:
                state = state.reshape((1,5,186,150))
            '''
            #x = self.main(state)
            #print(state.shape)
            inps1 = state
            #inps2 = helper
            action_m = self.layer1(inps1)
            #inps = torch.cat((action_m,inps2),1)
            #x = self.layer2(inps)
            
            action_mean = self.actor(action_m)
            action_log_std = self.action_log_std.expand_as(action_mean)
            #action_var = self.var(x) + 1
            #action_std = self.std(state)
            #print(action_std)
            #print(torch.mean(action_std))
            #self.set_action_std(torch.mean(action_std))
            #var = torch.mean(action_std)
            #print(action_mean.shape)
            #print(action_mean.shape,action_var.shape)
            #action_var = action_var.expand_as(action_mean)
            #print(action_var,action_var.shape)
            #print(action_mean.shape,action_var.shape)
            #cov_mat = torch.diag_embed(action_var)
            #print(cov_mat)
            dist = Normal(action_mean,action_log_std.exp())
            #dist = Beta(action_mean, action_var)
            #print(action_mean.shape,cov_var.shape)
            #dist = MultivariateNormal(action_mean,cov_mat)
            #print(dist)
            # for single action continuous environments
            #if self.action_dim == 1:
            #    action = action.reshape(-1, self.action_dim)

        else:
            #inps1 = state[:,:-3]
            #inps2 = state[:,-3:]
            state = state.reshape(state.shape[0],1,state.shape[1])
            action_m,_ = self.layer1(state)
            #inps = torch.cat((action_m,inps2),1)
            #
            bsize = state.shape[0]
            action_m = self.layer2(action_m.reshape(bsize,action_m.shape[-1]))
            action_probs = self.actor(action_m)
            dist = Categorical(action_probs)
        #action = (action + 1)/2
        action_logprobs = dist.log_prob(action)
        #print(action_logprobs.shape)
        dist_entropy = dist.entropy()
        inps1 = state
        #nps2 = helper
        critic_m,_ = self.layer1(inps1)
        #inps = torch.cat((critic_m,inps2),1)
        #action_mean = self.actor(inps)
        critic_m = self.layer2(critic_m.reshape(bsize,critic_m.shape[-1]))
        state_values = self.critic(critic_m)
        #print(state_values,action_mean)
        return action_logprobs, state_values, dist_entropy


In [ ]:
class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std_init=0.6):

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_std = action_std_init
        self.entropy_c = 0.01
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.layer1.parameters(), 'lr': lr_actor},
                        {'params': self.policy.layer2.parameters(), 'lr': lr_actor},
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                        #{'params': self.policy.action_log_std, 'lr': 0.01}
                        
                        
                    ])
        

        self.policy_old = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def set_action_std(self, new_action_std):
        
        if self.has_continuous_action_space:
            self.action_std = new_action_std
            self.policy.set_action_std(new_action_std)
            self.policy_old.set_action_std(new_action_std)
        
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling PPO::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def decay_action_std(self, action_std_decay_rate, min_action_std):
        #print("--------------------------------------------------------------------------------------------")

        if self.has_continuous_action_space:
            self.action_std = self.action_std - action_std_decay_rate
            self.action_std = round(self.action_std, 4)
            if (self.action_std <= min_action_std):
                self.action_std = min_action_std
                #print("setting actor output action_std to min_action_std : ", self.action_std)
            else:
                pass
                #print("setting actor output action_std to : ", self.action_std)
            self.set_action_std(self.action_std)

        else:
            print("WARNING : Calling PPO::decay_action_std() on discrete action space policy")

        #print("--------------------------------------------------------------------------------------------")


    def select_action(self, state,val = False):

        if self.has_continuous_action_space:
            with torch.no_grad():
                #helper = torch.FloatTensor(state[1]).to(device)
                state = torch.FloatTensor(state).to(device)
                
                action, action_logprob,mem = self.policy_old.act(state,val)
            if not val:
                self.buffer.states.append(state)
                #self.buffer.helper.append(helper)
                self.buffer.actions.append(action)
                self.buffer.logprobs.append(action_logprob)

            return action.detach().cpu().numpy().flatten(),mem

        else:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob = self.policy_old.act(state,val)
            if not val:
                self.buffer.states.append(state)
                self.buffer.actions.append(action)
                self.buffer.logprobs.append(action_logprob)

            return action.item()


    def update(self):
        global tester,te2
        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        #print(self.buffer.rewards)
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        #print("I",rewards)
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        #print(rewards,rewards.mean(),rewards.std(unbiased = False))
        rewards = (rewards - rewards.mean()) / (rewards.std(unbiased = False) + 1e-7)
        #print(rewards)
        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)
        #old_helper = torch.squeeze(torch.stack(self.buffer.helper, dim=0)).detach().to(device)
        batch_size = 32
        tester = old_states
        print(old_states.shape,old_actions.shape,old_logprobs.shape)
        #mem = (torch.zeros(1,old_states.shape[0],128).to(device),torch.zeros(1,old_states.shape[0],128).to(device))
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):
            #for index in BatchSampler(SubsetRandomSampler(range(len(self.buffer.states))), batch_size, False):
            # Evaluating old actions and values
                #print(index,old_actions[index],old_actions)
            #print(old_states[index].shape)
            #te2 = index
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            #temp_size = logprobs.shape[0]
            #print(state_values.shape)
            #print(logprobs.shape,logprobs)
            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            #print(state_values.shape)
            #state_values = torch.
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())
            #print(ratios)
            # Finding Spurrogate Loss
            #print(rewards[index].shape)
            advantages = rewards - state_values.detach()   
            #print(rewards)
            #print(ratios.shape,advantages.shape)
            #advantages = advantages.reshape(temp_size,1)
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            #print(state_values)
            #print(state_values.shape)
            # final loss of clipped objective PPO
            #print(rewards.shape,state_values.shape)
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            #print(surr1,surr2,state_values,rewards,dist_entropy,loss)
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            #self.optimizer2.step()
        if self.entropy_c > 0.001:
            self.entropy_c -= 0.0001
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))

In [ ]:
update_timestep = 1000     # update policy every n timesteps
K_epochs = 40             # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0001      # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 0 

In [ ]:
def prepare_ngsim_scenario(client: carla.Client, data_mode = "train") -> Scenario:
    data_dir = os.environ.get("NGSIM_DIR")
    #data_dir = os.listdir('/home/surender/Downloads/NGSIM')
    assert data_dir, "Path to the directory with NGSIM dataset is required"
    ngsim_map = NGSimDatasets.list()
    ngsim_dataset = ngsim_map[1]
    client.load_world(ngsim_dataset.carla_map.level_path)
    if data_mode == "train":
        return NGSimLaneChangeScenario(
            ngsim_dataset,
            dataset_mode=DatasetMode.TRAIN,
            data_dir=data_dir,
            reward_type=RewardType.DENSE,
            client=client,
        )
    else:
        return NGSimLaneChangeScenario(
            ngsim_dataset,
            dataset_mode=DatasetMode.VALIDATION,
            data_dir=data_dir,
            reward_type=RewardType.DENSE,
            client=client,
        )
def prepare_ego_vehicle(world: carla.World) -> carla.Actor:
    car_blueprint = world.get_blueprint_library().find("vehicle.audi.a2")

    # This will allow external scripts like manual_control.py or no_rendering_mode.py
    # from the official CARLA examples to take control over the ego agent
    car_blueprint.set_attribute("role_name", "hero")

    # spawn points doesnt matter - scenario sets up position in reset
    ego_vehicle = world.spawn_actor(
        car_blueprint, carla.Transform(carla.Location(0, 0, 500), carla.Rotation())
    )

    assert ego_vehicle is not None, "Ego vehicle could not be spawned"

    # Setup any car sensors you like, collect observations and then use them as input to your model
    return ego_vehicle
def cmd_carla():
    os.system("DISPLAY= /home/surender/Downloads/carlaOld/CarlaUE4.sh -benchmark -fps=10 -quality-level=Low -opengl -Resx=4 -Resy=4 -NoVSync")

In [ ]:
thp = threading.Thread(target = cmd_carla)
thp.start()


In [ ]:
host = "localhost"
port = 2000
client = carla.Client(host,port)

In [ ]:
scenario = prepare_ngsim_scenario(client)

In [ ]:
world = client.get_world()
spectator = world.get_spectator()
ego_vehicle = prepare_ego_vehicle(world)


In [ ]:

scenario.reset(ego_vehicle)
c = world.tick()

In [ ]:
scenario._lane_change_instants[104]

In [ ]:
p = PPO(1,2,lr_actor,lr_critic,gamma,K_epochs,eps_clip,False)

In [ ]:
total_reward_list = []
epoch_list = []
step_list = []

In [ ]:
def find_front_back(y_origin):
    d = scenario._ngsim_vehicles_in_carla._vehicle_by_vehicle_id
    y_left = y_origin - 2
    y_right = y_origin + 2
    x = ego_vehicle.get_location().x
    
    
    #Find closest front
    min_dist = 10000
    min_id = -1
    
    for i in d:
        x_cor = d[i].get_location().x
        y_cor = d[i].get_location().y
        if x_cor > x and y_cor>y_left and y_cor<y_right:
            diff = x_cor - x
            if diff<min_dist:
                min_dist = diff
                min_id = i
    #Find closest back
    minb_dist = 10000
    minb_id = -1
    for i in d:
        x_cor = d[i].get_location().x
        y_cor = d[i].get_location().y
        if x_cor < x and y_cor>y_left and y_cor<y_right:
            diff =  x - x_cor
            if diff<minb_dist:
                minb_dist = diff
                minb_id = i
    return min_id,minb_id
def prep_neighbours(current_frame):
    d = scenario._ngsim_vehicles_in_carla._vehicle_by_vehicle_id
    start_lane = find_front_back(scenario._start_lane_waypoint.transform.location.y)
    target_lane = find_front_back(scenario._target_lane_waypoint.transform.location.y)
    data = []
    df = scenario._ngsim_recording._df_by_timeslot[scenario._lane_change.timeslot]
    cf = df[df['Frame ID'] == current_frame]
    sf = cf[cf["Lane Identification"] == scenario._lane_change.lane_from]
    srf = cf[cf["Lane Identification"] == scenario._lane_change.lane_to]
    default_spd = scenario._veh.speed/1.99
    for i in (start_lane):
        try:
            x = d[i].get_location().x - ego_vehicle.get_location().x
            y = d[i].get_location().y - ego_vehicle.get_location().y
        except:
            x = 0
            y = 0 
        #print("Start:",sf[sf["Vehicle ID"] == i]["Vehicle Velocity"])
        try:
            spd = float(sf[sf["Vehicle ID"] == i]["Vehicle Velocity"])
        except:
            spd = default_spd
        data.append(x/100)
        data.append(y/100)
        data.append(spd/130)
    
    for i in (target_lane):
        try:
            x = d[i].get_location().x - ego_vehicle.get_location().x
            y = d[i].get_location().y - ego_vehicle.get_location().y
        except:
            x = 0
            y = 0
        #print(srf[srf["Vehicle ID"] == i]["Vehicle Velocity"])
        try:
            spd = float(srf[srf["Vehicle ID"] == i]["Vehicle Velocity"])
        except:
            spd = default_spd
        data.append(x/100)
        data.append(y/100)
        data.append(spd/130)
    return data

In [ ]:
birdview_producer = BirdViewProducer(
    client,  # carla.Client
    target_size=PixelDimensions(width=186, height=150),
    pixels_per_meter=4
    #crop_type=BirdViewCropType.FRONT_AREA_ONLY
    )

In [ ]:
p.buffer.clear()

In [ ]:
p.load("NonCommBestModel.mdl")

In [ ]:

pr = validate()
p.buffer.clear()

In [ ]:
p.policy_old.layer1.eval()
p.policy_old.layer2.eval()
p.policy_old.actor.eval()

In [ ]:
pr

In [ ]:
import pickle
f = open("NonCommGif/information.pkl",'wb')
pickle.dump(infor,f)
f.close()

In [ ]:
ste = []
for i in infor:
    ste.append(i[1])

In [ ]:
sum(ste)/len(ste)

In [ ]:
infor

In [ ]:
0.6875

In [ ]:
val_points = []

In [ ]:
for i in range(50):
    p.buffer.clear()   
    pr = validate()
    p.buffer.clear()
    val_points.append(pr)

In [ ]:
sum(val_points)/len(val_points)

In [ ]:
min(val_points)

In [ ]:
max(val_points)

In [ ]:
import pickle
f = open("ValNonCommAllRandom.pkl",'wb')
pickle.dump(val_points,f)
f.close()

In [ ]:

section = 5
epochs = 5000
freq = 5
freq_n = 3
update_freq = 100
decay_freq = 1000
decay_c = 1000
#val = 0
min_r_avg = -1
steer_w = 1
min_steer_w = 0
steer_decay = 0.1
mn=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
last_epoch = -1

victory = 0
victory_buffer = 0
buffer_limit = 5000
for epoch in range(1601,epochs):
    step = 0
    
    #scenario = prepare_ngsim_scenario(client)
    #world = client.get_world()
    #ego_vehicle = prepare_ego_vehicle(world)
    #scenario.reset(ego_vehicle)
    #c = world.tick()
    #print(len(list(world.get_actors())))
    #del ego_vehicle
    ids = list(world.get_actors())[0]
    carla.command.DestroyActor(ids)
    '''
    for i in list(world.get_actors()):
        if type(i) != carla.libcarla.Vehicle:
            ids = i.id
            carla.command.DestroyActor(ids)
    '''
    world = client.get_world()
    #spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)
    
    #scenario.close()
    #scenario = prepare_ngsim_scenario(client)
    
    #time.sleep(3)
    #world = client.get_world()
    #spectator = world.get_spectator()
    #client.reload_world()
    
    #ego_vehicle = prepare_ego_vehicle(world)
    
    scenario.reset(ego_vehicle)
    c = world.tick()
    
    ego_vehicle.apply_control(carla.VehicleControl(manual_gear_shift=True, gear=4))
    #ego_vehicle.apply_control(carla.VehicleControl(throttle=1.0, brake=1.0))
    c = world.tick()
    #ego_vehicle.apply_control(carla.VehicleControl(manual_gear_shift=False))
    #time.sleep(4)
    #ego_vehicle.apply_control(carla.VehicleControl(brake=0.0))
    try:
        cmd,reward,dp,_ = scenario.step(ego_vehicle)
    except:
        continue
    c = world.tick()
    
    #way = scenario._target_lane_waypoint.transform
    speed = scenario._veh.speed/1.99
    #c = world.tick()
    done = False
    total_r = 0
    val = 0
    way = scenario._target_lane_waypoint.transform
    target_boundary = scenario._target_lane_waypoint.transform
    ##left_y = target_boundary.location.y - 5
    #right_y = target_boundary.location.y + 5
    #initial_x 
    #way = way_cal(ego_vehicle,val)
    #print(scenario._sampled_dataset_excerpt_info)
    t_clip_n = 0.0
    t_clip_p = 1.0
    
    s_clip_n = -1.0
    s_clip_p = 1.0
    cmd_buffer = [0]
    yaw_buffer = [0]
    
    steer = 0
    steer_aug = 0.1
    prep_limit = 10*1
    pid = VehiclePIDController(ego_vehicle,0.7)
    lock = 0
    if victory_buffer > buffer_limit:
        section+=1
        victory_buffer = 0
        buffer_limit += buffer_limit
    while not done:
        '''
        while True:
            #print(current_frame,c)
            if current_frame >= c:
                #print(current_frame,c)
                break
        '''
        #speed = scenario._veh.speed/1.99
        way = scenario._target_lane_waypoint.transform
        way.location.x = ego_vehicle.get_location().x + 30
        #safe = check_safety(way)
        
        
        #print(safe)
        if step < prep_limit:
            speed = scenario._veh.speed/1.99
            k = pid.run_step(speed,way)
            #print(k.throttle)
            birdview = birdview_producer.produce(
                agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
                )
            ego_vehicle.apply_control(carla.VehicleControl(throttle=k.throttle, steer=np.clip(0.0,-1.0,1.0),brake=k.brake))
        else:
            if lock == 0:
                pid = VehiclePIDController(ego_vehicle,0.7)
                lock = 1
            birdview = birdview_producer.produce(
                agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
                )
            #a = birdview[0].reshape(1,186,150)
            #a = np.append(a,birdview[1].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[2].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[3].reshape(1,186,150),axis=0)
            #a# = np.append(a,birdview[4].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[4].reshape(1,224,224),axis=0)
            #rgb = BirdViewProducer.as_rgb(birdview)/255.
            #in_data = a.reshape(1,5,186,150)
            '''
            for i in range(3):
                rgb[:,:,i] = (rgb[:,:,i] - mn[i])/std[i]
            '''
            #in_data = rgb.reshape(1,3,224,224)
            '''
            with torch.no_grad():
                in_data = torch.FloatTensor(in_data).to(device)
                in_data = mobilenet.features(in_data)
                in_data = nn.AvgPool2d(7,7)(in_data)
                in_data = nn.Flatten()(in_data)
                in_data = in_data.detach().cpu().numpy()
            '''
            
            #in_data = mdl(in_data).numpy()
            #in_data = in_data.reshape(in_data.shape[0],62720)
            #in_data = in_data.reshape((1,5,186,150))
            #in_data = input_data.reshape((1,3,320,320))
            #print(in_data.shape)
            if val == 0 or val == 1:
                inps= [0,1,0,speed/130]
            if val == 2 or val == 5:
                inps = [0,0,1,speed/130]
            if val == 4 or val ==3:
                inps = [1,0,0,speed/130]
            #inps = np.array(inps)
            data = prep_neighbours(_["ngsim_dataset"]["frame"])
            data = data + inps
            inputs = np.array(data).reshape(1,1,16)
            #ap = np.concatenate((in_data[0],inps)).reshape(1,1283)

            action = p.select_action(inputs)
            #print(action)
            #print(action)
            #action[0] = action[0]*2 - 1
            #action[1] = action[1]*120
            '''
            if (val == 0  or val ==1):
                s_clip_n = -0.15
                s_clip_p = 0.15
                t_clip_n = 0.4
                t_clip_p = 1.0

            if (val == 2 or val == 5):
                s_clip_n = 0.25
                s_clip_p = 0.8
                t_clip_n = 0.0
                t_clip_p = 0.4

            if (val == 3 or val == 4):
                s_clip_n = -0.8
                s_clip_p = -0.25
                t_clip_n = 0.0
                t_clip_p = 0.4
            '''


            #t_clip_n = 0.0
            #t_clip_p = 0.7

            #s_clip_n = -0.6
            #s_clip_p = 0.6  
            #d_action = action_space[action]
            #steer_s = action[0]*0.1
            #steer += steer_s
            '''
            if action[0]<0:
                steer -= steer_aug
            elif action[0]>0:
                steer += steer_aug
            
            if steer < -0.1:
                steer = -0.1
            if steer > 0.1:
                steer = 0.1
            '''
            #speed = action[0]*80+20
            #speed = action[1]
            #speed = math.sqrt(speed**2)
            #_speed = np.clip(action[0], -1,1)
            #speed = ((_speed + 1)/2)*50 + 10
            
            
            brake = 0
            throttle = 0
            steer = 0
            steer = 0
            
            if action == 0:
                x = 15
                #speed = speed - 1
            if action == 1:
                x = 30
                #speed = speed - 1
            if action == 2:
                x = 50 
                #speed = speed - 1
            if action == 3:
                x = 75
                #speed = speed - 1
            if action == 4:
                x = 15
                speed = speed - 1
            if action == 5:
                x = 30
                speed = speed - 1
            if action == 6:
                x = 50
                speed = speed - 1
            if action == 7:
                x = 75
                speed = speed - 1

            if action == 8:
                x = 15
                speed = speed + 1
            if action == 9:
                x = 30
                speed = speed + 1
            if action == 10:
                x = 50
                speed = speed + 1
            if action == 11:
                x = 75
                speed = speed + 1
            '''
            if action == 0:
                x = 5
                speed = speed - 1
            if action == 1:
                x = 15
                speed = speed - 1
            if action == 2:
                x = 25 
                speed = speed - 1
            if action == 3:
                x = 35
                speed = speed - 1
            if action == 4:
                x = 45
                speed = speed - 1
            if action == 5:
                x = 55
                speed = speed - 1
            if action == 6:
                x = 65
                speed = speed - 1
            if action == 7:
                x = 75
                speed = speed - 1
            if action == 8:
                x = 85
                speed = speed - 1
            if action == 9:
                x = 95
                speed = speed -1
            if action == 10:
                x = 95
                speed = speed + 1
            if action == 11:
                x = 85
                speed = speed + 1
            if action == 12:
                x = 75
                speed = speed + 1
            if action == 13:
                x = 65
                speed = speed + 1
            if action == 14:
                x = 55
                speed = speed + 1
            if action == 15:
                x = 45
                speed = speed + 1
            if action == 16:
                x = 35
                speed = speed + 1
            if action == 17:
                x = 25
                speed = speed + 1
            if action == 18:
                x = 15
                speed = speed + 1
            if action == 19:
                x = 5
                speed = speed + 1
            '''
            #x = (action[0] + 1)/2
            #x = x*100
            #y = (action[1] + 1)/2
            #y = y*130
            way.location.x =  ego_vehicle.get_location().x + x
            #speed = y
            #ay.location.x =  ego_vehicle.get_location().x + x
            #way.location.y = ego_vehicle.get_location().y + y
            '''
            brake = 0
            throttle = 0
            if action[0] <0:
                brake = action[0]
                throttle = 0
            else:
                throttle = action[0]
                brake = 0
            '''
            '''
            if epoch > 400:
                if (val == 0  or val ==1):
                    s_clip_n = -0.15
                    s_clip_p = 0.15
                    t_clip_n = 0.4
                    t_clip_p = 1.0

                if (val == 2 or val == 5):
                    s_clip_n = 0.25
                    s_clip_p = 0.6
                    t_clip_n = 0.0
                    t_clip_p = 0.4

                if (val == 3 or val == 4):
                    s_clip_n = -0.6
                    s_clip_p = -0.25
                    t_clip_n = 0.0
                    t_clip_p = 0.4
            '''
            #if epoch < 20:
            #print(throttle,steer,brake)
            #pid = VehiclePIDController(ego_vehicle)

            k = pid.run_step(speed,way)
            #throttle = k.throttle
            #brake = k.brake

            avg_steer = k.steer 
            brake = k.brake
            throttle = k.throttle
            #print("SteerDiff:",abs(steer-k.steer),k.steer,steer)
            #print(throttle,steer,brake)
            #avg_steer= k.steer
            ego_vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=np.clip(avg_steer,-1.0,1.0),brake=brake))

        #ego_vehicle.apply_control(carla.VehicleControl(throttle=np.clip(throttle, t_clip_n, t_clip_p), steer=np.clip(action[1], s_clip_n, s_clip_p)))#,brake=np.clip(brake, 0.0, 1.0)))
        
        
        #print(steer,speed)
        try:
            cmd, reward, done, _ = scenario.step(ego_vehicle)
        except Exception as e:
            print(e)
            print("ERROR")
            reward = -1
            done = True
        #reward = reward/2
        
        '''
        if reward < 0 :
            reward = -0.1
        '''
        
        val = cmd.value
        #way = way_cal(ego_vehicle,val)
        cmd_buffer.append(val)
        yaw_buffer.append(ego_vehicle.get_transform().rotation.yaw)
        if section == 0:
            if len(cmd_buffer) > 10:
                if _['on_target_lane']:
                    reward = 1 
                    done = True
        if section == 1:
            if len(cmd_buffer) > 10:
                if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and total_r>=0.5:
                    reward = 1 
                    done = True
        if section == 2:
            if len(cmd_buffer) > 10:
                if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10 and total_r>=0.5:
                    reward = 1 
                    done = True
        if section==3:
            if len(cmd_buffer) > 10:
                if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10 and total_r>=0.6:
                    reward = 1 
                    done = True
        if section==4:
            if len(cmd_buffer) > 10:
                if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=5 and total_r>=0.8:
                    reward = 1 
                    done = True
        if section>=5:
            pass
        if total_r >=0.9:
            print("Close to win")
        if reward >= 1:
            victory_buffer+=1
            print("Its a win")
            
        
            #p.save("WinModel"+str(epoch)+".mdl")
        #way = _["scenario_data"]["original_veh_transform"]
        #way = scenario._target_lane_waypoint.transform  
        
        
        #print(action[0],action[1])
        #print(done)
        #if done:
        #    print(_)
        #print(_)
        '''
        if (val == 0  or val ==1):
            if action[0] > 0.0:
                reward += 0.01
            if action[1] < 0.1 and action[1]> -0.1:
                reward += 0.01
                

        if (val == 2 or val == 5):
            if action[0] <0.1:
                reward += 0.01
            if action[1] > 0.0:
                reward += 0.01

        if (val == 3 or val == 4):
            if action[0] < 0.1:
                reward += 0.01
            if action[1] < 0.0:
                reward += 0.01
        '''
        #reward += step
        '''
        v = ego_vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        
        if kmh < 60 & kmh > 0.2:
            #done = False
            reward += 1 #-1
            # Reward lighter steering when moving
            if np.abs(action[1]) < 0.3:
                reward += 1
            elif np.abs(action[1]) > 0.5 and np.abs(action[1]) < 0.9:
                reward -= 0.1
            elif np.abs(action[1]) >= 0.9:
                reward -= 0.2
        elif kmh < 0.2:
            reward -= 0.1
        else:
            #print("Maybe never")
            reward += 0.01
            if np.abs(action[1]) < 0.3:
                reward += 0.12
            # Reduce score for heavy steering
            if np.abs(action[1]) > 0.5 and np.abs(action[1]) < 0.9:
                reward -= 0.17
            elif np.abs(action[1]) >= 0.9:
                reward -= 0.21
        '''
        
        rgb = BirdViewProducer.as_rgb(birdview)
        cv2.imshow('Frame',rgb)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        
        if step < prep_limit:
            pass
        else:
            #print("HI")
            p.buffer.rewards.append(reward)
            p.buffer.is_terminals.append(done)
        
        total_r += reward
        step += 1
        
        '''
        if step % freq ==0 :
            print(step)
            p.update()
        
        if step % freq_n == 0:
            p.decay_action_std(0.05,0.1)
        
        
        '''
        world.tick()
    #total_r += step/10
    '''
    if epoch > 50:
        
        if sum(total_reward_list[-50:])/len(total_reward_list[-50:]) > min_r_avg or sum(step_list) > decay_freq:
            decay_freq += decay_c
            #decay_c -= 50
            #print("Decaying:",p.action_std)
            p.decay_action_std(0.01,0.1)
            
            min_r_avg = sum(total_reward_list[-50:])/len(total_reward_list[-50:])
    '''
    if epoch%update_freq ==0 and epoch != 0 and len(p.buffer.states)>1:
            print(k.steer)
            print("Update with batches:",len(p.buffer.states))
            p.update()
            vs = validate()
            val_points.append(vs)
            print(vs)
            p.buffer.clear()
            #update_freq = 10
            if steer_w > min_steer_w +0.1:
                steer_w -= steer_decay
    '''
    else:
            print(total_r,epoch,step)
            total_reward_list.append(total_r)
            epoch_list.append(epoch)
            step_list.append(step)
            continue
    #except Exception as e:
        #print("Error:",e)
        #pass
    '''
    if step <10 and epoch != 0 and epoch!=last_epoch+1:
        last_epoch = epoch
        scenario = prepare_ngsim_scenario(client)
        world = client.get_world()
        ego_vehicle = prepare_ego_vehicle(world)
        scenario.reset(ego_vehicle)
        world.tick()
    cv2.destroyAllWindows()
    print(total_r,epoch,step,"Section:",section,"V_Buffer:",victory_buffer)
    total_reward_list.append(total_r)
    epoch_list.append(epoch)
    step_list.append(step)
    

In [ ]:
iter_rgb = {}
infor = []

In [ ]:
max_score = 0.80

In [ ]:

def validate():
    global max_score
    val_success = []
    try:
        scenario.close()
        #self.scenario_val.close()
    except:
        pass
    
    scenario = prepare_ngsim_scenario(client,"Val")
    world = client.get_world()
    ego_vehicle = prepare_ego_vehicle(world)
    birdview_producer = BirdViewProducer(
            client,  # carla.Client
            target_size=PixelDimensions(width=186, height=150),
            pixels_per_meter=4
            #crop_type=BirdViewCropType.FRONT_AREA_ONLY
    )
    
    #self.scenario_val.reset(self.ego_vehicle)
    scenario.reset(ego_vehicle,0)
    world.tick()
    val_count = 0
    for i in range(100):
        temp_rgb = []
        t_clip_n = 0.0
        t_clip_p = 1.0

        s_clip_n = -1.0
        s_clip_p = 1.0
        #torch.cuda.empty_cache()
        step = 0
        #del self.ego_vehicle
        #self.scenario = prepare_ngsim_scenario(self.client,"Val")
        ids = list(world.get_actors())[0].id
        carla.command.DestroyActor(ids)
        world = client.get_world()
        #self.spectator = self.world.get_spectator()
        ego_vehicle = prepare_ego_vehicle(world)
        
        scenario.reset(ego_vehicle,i)
        c = world.tick()
        '''
        cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
        cam_bp.set_attribute("image_size_x",str(320))
        cam_bp.set_attribute("image_size_y",str(320))
        cam_bp.set_attribute("fov",str(100))
        cam_location = carla.Location(-5,0,4)
        cam_rotation = carla.Rotation(0,0,0)
        cam_transform = carla.Transform(cam_location,cam_rotation)
        ego_front_cam = world.spawn_actor(cam_bp,cam_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
        #self.rgb_front_listener = ego_cam
        ego_front_cam.listen(lambda image: check_img(image,i))
        '''
        ego_vehicle.apply_control(carla.VehicleControl(manual_gear_shift=True, gear=4))
        c = world.tick()
        try:
            cmd,reward,dp,_ = scenario.step(ego_vehicle)
        except Exception as e:
            print(e)
            continue
        c = world.tick()
        speed = scenario._veh.speed/1.99

        '''
        except Exception as e:
            print("Error in val init:",e)
            val_success.append(0)
            self.load_carla()
            continue
        '''

        done = False
        reward = 0
        val = 0
        #way = self.way_cal(self.ego_vehicle,val)
        way = scenario._target_lane_waypoint.transform
        cmd_buffer = [0]
        yaw_buffer = [0]
        total_rew = 0
        target_buffer = [False]
        steer = 0
        steer_aug = 0.05
        prep_limit = 10*1
        pid = VehiclePIDController(ego_vehicle,0.7)
        lock = 0
        fin = 0
        while not done:
                birdview = birdview_producer.produce(
                agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
                )
                #speed = scenario._veh.speed/1.99
                way = scenario._target_lane_waypoint.transform
                way.location.x = ego_vehicle.get_location().x + 40
                #safe = self.check_safety(way)
                '''
                while True:
                    #print(current_frame,c)
                    if current_frame >= c:
                        #print(current_frame,c)
                        break
                '''
                if step < prep_limit:
                    speed = scenario._veh.speed/1.99
                    k = pid.run_step(speed,way)
                    #print(k.throttle)
                    ego_vehicle.apply_control(carla.VehicleControl(throttle=k.throttle, steer=np.clip(0.0,-1.0,1.0),brake=k.brake))
                else:
                    if lock == 0:
                        pid = VehiclePIDController(ego_vehicle,0.7)
                        lock = 1
                    #birdview = self.birdview_producer.produce(
                    #    agent_vehicle=self.ego_vehicle  # carla.Actor (spawned vehicle)
                    #    )
                    #a = birdview[0].reshape(1,50,150)
                    #a = np.append(a,birdview[1].reshape(1,50,150),axis=0)
                    # = np.append(a,birdview[2].reshape(1,50,150),axis=0)
                    ##a = np.append(a,birdview[3].reshape(1,50,150),axis=0)
                    #a = np.append(a,birdview[4].reshape(1,50,150),axis=0)
                    #a = np.append(a,birdview[4].reshape(1,224,224),axis=0)
                    '''
                    rgb = BirdViewProducer.as_rgb(birdview)/255.
                    for i in range(3):
                        rgb[:,:,i] = (rgb[:,:,i] - self.mn[i])/self.std[i]
                    #in_data = a.reshape(1,5,186,150)
                    in_data = rgb.reshape(1,3,224,224)

                    with torch.no_grad():
                        in_data = torch.FloatTensor(in_data).to(device)
                        in_data = self.mobilenet.features(in_data)
                        in_data = nn.AvgPool2d(7,7)(in_data)
                        in_data = nn.Flatten()(in_data)
                        in_data = in_data.detach().cpu().numpy()
                    '''
                    #in_data = a.reshape(1,5,50,150)
                    if val == 0 or val == 1:
                        inps= [0,1,0,speed/130]
                    if val == 2 or val == 5:
                        inps = [0,0,1,speed/130]
                    if val == 4 or val ==3:
                        inps = [1,0,0,speed/130]
                    #inps = np.array(inps)
                    #ap = np.concatenate((in_data[0],inps)).reshape(1,1283)
                    #inputs = [in_data,np.array(inps).reshape(1,4)]
                    data = prep_neighbours(_["ngsim_dataset"]["frame"])
                    data = data + inps
                    #inps = np.array(inps)
                    #ap = np.concatenate((in_data[0],inps)).reshape(1,1283)
                    #inputs = [in_data,np.array(inps).reshape(1,4)]
                    inputs = np.array(data).reshape(1,1,16)
                    with torch.no_grad():
                        action = p.select_action(inputs,True)
                    #action = action * 2 - 1
                    steer = 0
                    #x = (action[0] + 1)/2
                    #x = x*100
                    #y = action[1]*5
                    #y = (action[1] +1)/2
                    #y = y*130
                    if action == 0:
                        x = 15
                        #speed = speed - 1
                    if action == 1:
                        x = 30
                        #speed = speed - 1
                    if action == 2:
                        x = 50 
                        #speed = speed - 1
                    if action == 3:
                        x = 75
                        #speed = speed - 1
                    if action == 4:
                        x = 15
                        speed = speed - 1
                    if action == 5:
                        x = 30
                        speed = speed - 1
                    if action == 6:
                        x = 50
                        speed = speed - 1
                    if action == 7:
                        x = 75
                        speed = speed - 1
                    
                    if action == 8:
                        x = 15
                        speed = speed + 1
                    if action == 9:
                        x = 30
                        speed = speed + 1
                    if action == 10:
                        x = 50
                        speed = speed + 1
                    if action == 11:
                        x = 75
                        speed = speed + 1
                    
                    '''
                    speed_var = 0.5
                    if action == 0:
                        x = 5
                        speed = speed - speed_var
                    if action == 1:
                        x = 15
                        speed = speed - speed_var
                    if action == 2:
                        x = 25 
                        speed = speed - speed_var
                    if action == 3:
                        x = 35
                        speed = speed - speed_var
                    if action == 4:
                        x = 45
                        speed = speed - speed_var
                    if action == 5:
                        x = 55
                        speed = speed - speed_var
                    if action == 6:
                        x = 65
                        speed = speed - speed_var
                    if action == 7:
                        x = 75
                        speed = speed - speed_var
                    if action == 8:
                        x = 85
                        speed = speed - speed_var
                    if action == 9:
                        x = 95
                        speed = speed -speed_var
                    if action == 10:
                        x = 95
                        speed = speed + speed_var
                    if action == 11:
                        x = 85
                        speed = speed + speed_var
                    if action == 12:
                        x = 75
                        speed = speed + speed_var
                    if action == 13:
                        x = 65
                        speed = speed + speed_var
                    if action == 14:
                        x = 55
                        speed = speed + speed_var
                    if action == 15:
                        x = 45
                        speed = speed + speed_var
                    if action == 16:
                        x = 35
                        speed = speed + speed_var
                    if action == 17:
                        x = 25
                        speed = speed + speed_var
                    if action == 18:
                        x = 15
                        speed = speed + speed_var
                    if action == 19:
                        x = 5
                        speed = speed + speed_var

                    '''
                    
                    way.location.x =  ego_vehicle.get_location().x + x
                    #way.location.y = self.ego_vehicle.get_location().y + y
                    #speed = y
                    brake = 0
                    throttle = 0
                    #_speed = np.clip(action[0], -1,1)
                    #speed = ((_speed + 1)/2)*50 + 10



                    #pid = VehiclePIDController(self.ego_vehicle)
                    k = pid.run_step(speed,way)
                    throttle = k.throttle
                    brake = k.brake

                    avg_steer = k.steer
                    ego_vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=np.clip(avg_steer, s_clip_n, s_clip_p),brake=brake))

                #ego_vehicle.apply_control(carla.VehicleControl(throttle=np.clip(throttle, t_clip_n, t_clip_p), steer=np.clip(action[1], s_clip_n, s_clip_p)))#,brake=np.clip(brake, 0.0, 1.0)))


                try:

                    cmd, reward, done, _ = scenario.step(ego_vehicle)
                except Exception as e:
                    print(e)
                    break
                '''
                except:
                    break

                if reward < 0 :
                    reward = -0.1
                '''
                '''
                if reward <= -1:
                    reward = 0
                '''

                #print(reward, cmd)

                val = cmd.value
                cmd_buffer.append(val)
                yaw_buffer.append(ego_vehicle.get_transform().rotation.yaw)
                target_buffer.append(_['on_target_lane'])
                #fin = 0
                if len(cmd_buffer) > 10:
                    if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10 and total_rew>=0.7:
                        reward = 1
                        done = True
                        pass
                '''
                if len(cmd_buffer) > 10:
                    if _['on_target_lane'] and all(item == True for item in target_buffer[-10:]):
                        reward = 1 + random.random()
                        done = True

                '''
                #rgb = BirdViewProducer.as_rgb(birdview)
                #temp_rgb.append(rgb)
                total_rew += reward
                #total_rew += reward
                #way = self.way_cal(self.ego_vehicle,val)
                #way = scenario._target_lane_waypoint.transform 
                c = world.tick()
                step+=1
        #iter_rgb[i] = temp_rgb
        if step>1:
            val_count+=1
        if total_rew >= 0.0:
            #print("Success!")
            #val_reward.append(reward)
            #val_count += 1
            
            if fin == 1:
                total_rew += 1.0
            val_success.append(1)
        else:
            #val_reward.append(reward)
            #if total_rew+1.0 >=0:
            #    val_success.append(total_rew+1.0)
            #else:
            val_success.append(0)
        #infor.append([i,step,total_rew])
        #print(total_rew)
    print(sum(val_success)," <-- Number of success")
    try:
        scenario.close()
    except:
        pass
    scenario = prepare_ngsim_scenario(client)
    world = client.get_world()
    #self.spectator = self.world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)
    birdview_producer = BirdViewProducer(
        client,  # carla.Client
        target_size=PixelDimensions(width=186, height=150),
        pixels_per_meter=4
        #crop_type=BirdViewCropType.FRONT_AREA_ONLY
    )

    '''
    except Exception as e:
        print("Re init Train:",e)
        self.load_carla()
        self.scenario = prepare_ngsim_scenario(self.client)
        self.world = self.client.get_world()
        #self.spectator = self.world.get_spectator()
        self.ego_vehicle = prepare_ego_vehicle(self.world)
        self.birdview_producer = BirdViewProducer(
            self.client,  # carla.Client
            target_size=PixelDimensions(width=150, height=186),
            pixels_per_meter=4,
            crop_type=BirdViewCropType.FRONT_AREA_ONLY
        )
    '''
    scenario.reset(ego_vehicle)
    world.tick() 
    if val_count == 0:
        val_count = 1
    val_scores = sum(val_success)/(len(val_success))
    if val_scores > max_score:
        print("Found a max score!")
        max_score = val_scores
        p.save("BestScoreNModel"+str(int(max_score))+".mdl")
    return val_scores

In [ ]:
p.buffer.clear()

In [ ]:
p.save("SmallHalfModel.mdl")

In [ ]:
len(total_reward_list)

In [ ]:
total_reward_list

In [ ]:
iter_cam = {}
for i in range(50):
    iter_cam[i] = []

def check_img(img,i):
    array = np.frombuffer(img.raw_data, dtype=np.dtype("uint8"))
    #print(array.shape)
    array = np.reshape(array, (img.height, img.width, 4)) # RGBA format
    array = array[:, :, :3] #  Take only RGB
    #print(array.shape)
    #plt.imshow(array)
    
    img = Image.fromarray(array)
    
    #print(img)
    img = img.resize((320,320), Image.ANTIALIAS)
    #print(img)
    input_data = np.array(img)
    iter_cam[i].append(input_data)

In [ ]:
ego_vehicle

In [ ]:
scenario.reset(ego_vehicle)

In [ ]:
cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
cam_bp.set_attribute("image_size_x",str(320))
cam_bp.set_attribute("image_size_y",str(320))
cam_bp.set_attribute("fov",str(100))
cam_location = carla.Location(2,0,20)
cam_rotation = carla.Rotation(-90,0,0)
cam_transform = carla.Transform(cam_location,cam_rotation)
ego_front_cam = world.spawn_actor(cam_bp,cam_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
#self.rgb_front_listener = ego_cam
ego_front_cam.listen(lambda image: check_img(image))

In [ ]:
train_score = []
for i in range(0,6000,100):
    c = i*100
    k = total_reward_list[i:i+100]
    count = 0
    for j in k:
        if j>0.0:
            count+=1
    train_score.append(count)        
    
    

In [ ]:
len(iter_cam)

In [ ]:
198,272

In [ ]:
val_points

In [ ]:
plt.imshow(iter_cam[61])

In [ ]:
success = []
for i in infor:
    if i[2]>0:
        success.append(i[0])

In [ ]:
for i in success:
    imageio.mimsave('Te'+str(i)+'.gif', iter_cam[i], fps=30)

In [ ]:
for i in iter_rgb:
    imageio.mimsave('NonCommGif/scene'+str(i)+'.gif', iter_rgb[i], fps=10)

In [ ]:
success

In [ ]:
infor

In [ ]:
iter_rgb.keys()

In [ ]:
iter_cam

In [ ]:
train_score

In [ ]:
val_points

In [ ]:
len(val_points)

In [ ]:
#plt.ylim(0,1)
plt.plot(list(range(len(val_points))),val_points)

In [ ]:
val_points

In [ ]:
print(p.policy)

In [ ]:
dt=p.select_action(,)

In [ ]:
for i in range(5):
    done = False
    #mem = (torch.zeros(1,1,128).to(device),torch.zeros(1,1,128).to(device))
    c = 0
    while not done:
        st = torch.rand(1,1,16)
        dt= p.select_action(st)
        print(dt)
        reward = 0.1
        if c >5:
            done = True
        c+=1
        p.buffer.rewards.append(reward)
        p.buffer.is_terminals.append(done)

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
p.update()

In [ ]:
tester[te2]

In [ ]:
te2

In [ ]:
dt[0]

In [26]:
x = torch.rand(1,16)

In [27]:
z = Categorical(nn.Softmax(dim=-1)(nn.Linear(16,4)(x))).sample()

In [28]:
c = z.reshape(1,1)

In [34]:

c = torch.FloatTensor(c).to('cpu')

TypeError: expected TensorOptions(dtype=float, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)) (got TensorOptions(dtype=long int, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)))

In [30]:
nn.Linear(1,5)(c)

RuntimeError: expected scalar type Float but found Long

In [ ]:
a = [random.random() for x in range(16)]

In [ ]:
with torch.no_grad():
    a = torch.FloatTensor(a)
    c = torch.FloatTensor([a,a,a,a])

In [ ]:
torch.zeros(1,32,128)

In [ ]:
inp = torch.cat((a, a, a, a, a), 0).reshape(1,5,16)

In [ ]:
len(scenario._lane_change_instants)

In [ ]:
cmd,r,done,_ = scenario.step(ego_vehicle)

In [ ]:
cmd.value

In [ ]:
import pickle
f = open("val_score2.pkl","wb")
pickle.dump(val_points,f)
f.close()

In [ ]:
import pickle
f = open("val_score2.pkl",'rb')
val_points = pickle.load(f)
f.close()

In [ ]:
import imageio

In [ ]:
for i in iter_rgb:
    print(len(i))

In [ ]:
infor

In [ ]:
imageio.mimsave('video1.gif', iter_cam[205:290], fps=10)

In [ ]:
len(iter_rgb)

In [ ]:
lane_dir 

In [ ]:
lane_dir = []

In [ ]:
len(lane_dir)

In [ ]:
import pickle
f = open("Data.pkl","wb")
pickle.dump(lane_dir,f)
f.close()

In [ ]:
import pickle
f = open("Data.pkl",'rb')
lane_dir = pickle.load(f)
f.close()

In [ ]:
ego_vehicle

In [ ]:
scenario.reset(ego_vehicle,0)


In [ ]:
blacklist = [103,223,519,560]

In [ ]:
for i in range(743,965):
    if i in blacklist:
        continue
    print(i)
    try:
        scenario.reset(ego_vehicle,i)
        world.tick()
        cmd,r,done,_ = scenario.step(ego_vehicle)
        world.tick()
        lane_dir.append(cmd.value)
    except:
        scenario = prepare_ngsim_scenario(client)
        world = client.get_world()
        spectator = world.get_spectator()
        ego_vehicle = prepare_ego_vehicle(world)
        scenario.reset(ego_vehicle,i)
        world.tick()
        cmd,r,done,_ = scenario.step(ego_vehicle)
        world.tick()
        lane_dir.append(cmd.value)

In [ ]:
a = set(range(1,10))
b = set([1,3])

In [ ]:
import random

In [ ]:
random.choice(list(set(range(965)).difference(set([103,223,519,560]))))

In [ ]:
a = list(range(1,10)) - [1,3]

In [ ]:
len(lane_dir)

In [ ]:
cd = [x - 4 for x in lane_dir]

In [ ]:
np.bincount(cd)

In [ ]:
cmd

In [ ]:
inp

In [ ]:
c = c.reshape(1,5,16)

In [ ]:
with torch.no_grad():
    in_data = torch.FloatTensor(c)

In [ ]:
in_data.shape

In [ ]:
ct = a.reshape(1,16)

In [ ]:
in_data

In [ ]:
p.layer1(ct)

In [ ]:
rnn = nn.LSTM(10, 20, 2)
inp = torch.randn(1, 1, 10)

In [ ]:
rnn(inp)[0].shape

In [ ]:
p.layer1(inp)